In [322]:
#!/usr/bin/env python

import re

#test file
sam_file = "/Users/GioiTran/Documents/shell/Bi624/Deduper/input_sorted_unit_test.sam"

#known barcode file
UMI_file = "/Users/GioiTran/Documents/shell/Bi624/Deduper/STL96.txt"

#output file 
output_file = "/Users/GioiTran/Documents/shell/Bi624/Deduper/output.sam"

def get_UMI(file):
    with open(UMI_file, "rt") as fh:
        known_UMI = []
        
        for line in fh:
            known_UMI.append(line.rstrip('\n'))
    return(known_UMI)

def parsing_QNAME(qname):
    qname_list = qname.split(":")
    #print(qname_list)
    return qname_list[-1] #get the last element in the list (where barcode is)

def forward(CIGAR_str, POS):
    cigar_list = re.findall('(\d+)([MNS])', CIGAR_str )
    
    if "S" in cigar_list[0]:
        fw_softclip = (cigar_list[0])
        
        adj_POS = (POS) - int(fw_softclip[0])
        return(adj_POS)
    if "S" not in cigar_list[0]:
        adj_POS = POS
        return(adj_POS)
         
    
cigar = "1S20M1D20I200D20M1S"
forward(cigar, 200)

def reverse(CIGAR_str, POS):
    cigar_list = re.findall('(\d+)([MNSD])', CIGAR_str )
    #print(cigar_list)
    if "S" in cigar_list[0]:
        del cigar_list[0]
        #print(cigar_list[0])
        for char in cigar_list:
            if "N" or "D" or "M" or "S" in char:
                POS = POS + int(char[0])
                #print(char[0])                
    else:
        for char in cigar_list:
            if "N" or "D" or "M" or "S" in char:
                POS = POS + int(char[0])
                #print(char[0])
    return(POS)
            
    
#testing functions
#cigar = "20M1D20I200D20M1S"
#forward(cigar, 200)
#reverse(cigar, 200)
#parsing_QNAME("NS500451:154:HWKTMBGXX:1:11101:15364:1139:GAACAGGT")

file_out=open(output_file, "a")

with open(sam_file, "rt") as file_handle:
    
    record_set_fw = set()
    record_set_rv = set()
    record_tuple = ()
    umi_list = get_UMI(UMI_file)
    previous_chromosome = 0
    
    print(len(umi_list))
    
    for line in file_handle:
    
        if "@" in line:
            file_out.write(line)
        else:
            ref_rec = line.split()
            
            curr_chromosome = ref_rec[2]
            #group sets by chromosome
            
            #if curr_chromosome != previous_chromosome:
              #  previous_chromosome = curr_chromosome
              #  print(previous_chromosome)
                #increment chromosome_counter
            
            
            if parsing_QNAME(ref_rec[0]) in umi_list and ((int(ref_rec[1]) & 16) != 16):
                
                adj_5prime_pos_fw = forward(ref_rec[5], int(ref_rec[3]))
                #add qname, chromosome, and adjusted 5' position into a tuple
                record_tuple_fw=  (ref_rec[0] ,ref_rec[2],adj_5prime_pos_fw)
                #print(record_tuple_fw)
                #record_set.add(record_tuple)
                #print(adj_5prime_pos)
                #print(record_set)
                
                if record_tuple_fw not in record_set_fw:
                    record_set_fw.add(record_tuple_fw)
                    
                    file_out.write(line)
                    print(record_set_fw)
                                    
            #account for adjusted position if bitwise flag is reverse    
            if parsing_QNAME(ref_rec[0]) in umi_list and ((int(ref_rec[1]) & 16) == 16):
                adj_5prime_pos_rv = reverse(ref_rec[5], int(ref_rec[3]))
                
                record_tuple_rv = (ref_rec[0], ref_rec[2], adj_5prime_pos_rv)
                #print(record_tuple_rv)
                
                #print(record_set)
                #print(ref_rec)
                
                if record_tuple_rv not in record_set_rv:
                    record_set_rv.add(record_tuple_rv)
                    file_out.write(line)
                

file_out.close()
            
           
            
        
                
            
                
      
        

        
    

96
{('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AACGCCAT', '1', 197)}
{('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AACGCCAT', '1', 197), ('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AAGGTACG', '1', 197)}
{('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AACGCCAT', '2', 199), ('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AACGCCAT', '1', 197), ('NS500451:154:HWKTMBGXX:1:11101:15364:1139:AAGGTACG', '1', 197)}
